In [ ]:
import face_recognition
import cv2
import numpy as np
import os

In [ ]:
BASE_DIR = "C:\\Users\\Nikhil kr Parashar\\face_rec"
image_dir = os.path.join(BASE_DIR,"face train")

In [ ]:
video_capture = cv2.VideoCapture(0)

In [ ]:
def get_encoded_faces():
    encoded = {}

    for dirpath, dnames, fnames in os.walk(image_dir):
        for f in fnames:
            if f.endswith(".jpg") or f.endswith(".png") or f.endswith(".jpeg") or f.endswith(".PNG") or f.endswith(".JPG") or f.endswith(".JPEG") :
                face = face_recognition.load_image_file("face train/" + f)
                encoding = face_recognition.face_encodings(face)[0]
                encoded[f.split(".")[0]] = encoding

    return encoded

In [ ]:
def classify_face():
    
    faces = get_encoded_faces()
    
    known_face_encodings = list(faces.values())

    known_face_names = list(faces.keys())

    # Initialize some variable
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True

    while True:
        ret, frame = video_capture.read()
        
        small_frame = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)

        rgb_small_frame = small_frame[:,:,::-1]
        
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        for encodeFace,faceLoc in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_face_encodings,encodeFace,0.5)
            faceDis = face_recognition.face_distance(known_face_encodings, encodeFace)
            matchIndex = np.argmin(faceDis)
            name ="Unknown"
            
            if matches[matchIndex]:
                name = known_face_names[matchIndex]
                y1,x2,y2,x1 = faceLoc
                y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0),2)
                cv2.rectangle(frame,(x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED)
                cv2.putText(frame, name, (x1-6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),2)
            if not matches[matchIndex]:
                y1,x2,y2,x1 = faceLoc
                y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0),2)
                cv2.rectangle(frame,(x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED)
                cv2.putText(frame, name, (x1-6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),2)


        # Display Result
        cv2.imshow("Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
classify_face()